In [387]:
import itertools

import numpy
import pandas
import scipy.stats as stats
import statsmodels.api as sm
import numpy as np

In [388]:
df = pandas.read_csv('real_estate_dataset.csv').drop(columns=['ID'])
target = 'Price'
df.insert(0, 'const', 1)
df

,const,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
0,1,143.635030,1,3,3,1967,1,1,48,8.297631,5.935734,602134.816747
1,1,287.678577,1,2,1,1949,0,1,37,6.061466,10.827392,591425.135386
2,1,232.998485,1,3,2,1923,1,0,14,2.911442,6.904599,464478.696880
3,1,199.664621,5,2,2,1918,0,0,17,2.070949,8.284019,583105.655996
4,1,89.004660,4,3,3,1999,1,0,34,1.523278,14.648277,619879.142523
...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,138.338057,2,2,2,1967,1,0,16,4.296086,5.562583,488496.350722
496,1,195.914028,2,3,1,1977,0,1,45,7.406261,2.845105,657736.921717
497,1,69.433659,1,1,2,2004,0,0,18,8.629724,6.263264,405324.950201
498,1,293.598702,5,1,3,1940,1,0,41,5.318891,16.990684,773035.968028


In [389]:
# df = pandas.get_dummies(df,
#                         columns=['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea',
#                                  'furnishingstatus'], dtype=int, drop_first=True)
# df

Задание 2

In [390]:
model = sm.OLS(df[target], df.drop(columns=[target])).fit()
print(f"Средняя ошибка аппроксимации: {np.mean(np.abs(model.resid) / df[target].to_numpy()) * 100:.3f}%")
model.summary()

Средняя ошибка аппроксимации: 2.760%


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     1987.
Date:                Wed, 26 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:09:46   Log-Likelihood:                -5633.8
No. Observations:                 500   AIC:                         1.129e+04
Df Residuals:                     489   BIC:                         1.134e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -2.897e+06   4.79e+04    -60.462      0.000   -2.99e+06    -2.8e+06
Square_Feet         1017.8034     11.602     87.726      0.000     995.007    1040.600
Num_Bedrooms        5.069e+04    598.496     84.688      0.000    4.95e+04    5.19e+04
Num_Bathrooms       2.995e+04   1052.637     28.450      0.000    2.79e+04     3.2e+04
Num_Floors          2.113e+04   1077.000     19.623      0.000     1.9e+04    2.33e+04
Year_Built          1518.2413     24.276     62.542      0.000    1470.544    1565.939
Has_Garden          3.046e+04   1728.274     17.622      0.000    2.71e+04    3.39e+04
Has_Pool            4.742e+04   1743.961     27.188      0.000     4.4e+04    5.08e+04
Garage_Size         1135.2777     74.744     15.189      0.000     988.419    1282.137
Location_Score      4805.3634    302.587     15.881      0.000    4210.832    5399.895
Distance_to_Center -1944.8982    156.369    -12.438      0.000   -2252.137   -1637.660
==============================================================================
Omnibus:                        0.036   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.982   Jarque-Bera (JB):                0.039
Skew:                          -0.019   Prob(JB):                        0.981
Kurtosis:                       2.980   Cond. No.                     1.10e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.1e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Задание 3

In [391]:
def check_multicollinearity(dataframe, output=True):
    n = len(dataframe)
    columns_to_drop = [x for x in ['const', target] if x in dataframe.columns]
    m = len(dataframe.columns) - len(columns_to_drop)
    if columns_to_drop:
        c_matrix = dataframe.drop(columns=columns_to_drop).corr(method='pearson')
    determinant = numpy.linalg.det(c_matrix)
    statistic = -(n - 1 - (2 * m + 5) / 6) * numpy.log(determinant)
    critical = stats.chi2.ppf(0.1, m * (m - 1) / 2)
    if output:
        print(f"Наблюдаемая статистика: {statistic:.3f}")
        print(f"Критическая статистика: {critical:.3f}")
        print(f"Мультиколлинеарность: {'присутствует' if statistic > critical else 'отсутствует'}")
    return statistic < critical


_ = check_multicollinearity(df)

Наблюдаемая статистика: 50.066
Критическая статистика: 33.350
Мультиколлинеарность: присутствует


In [392]:
import numpy as np
from sklearn.linear_model import LinearRegression


def partial_f_test(X, y, feature_to_test):
    model_full = LinearRegression().fit(X, y)
    ssr_full = np.sum((y - model_full.predict(X)) ** 2)
    df_full = X.shape[0] - X.shape[1] - 1

    X_reduced = X.drop(columns=[feature_to_test])
    model_reduced = LinearRegression().fit(X_reduced, y)
    ssr_reduced = np.sum((y - model_reduced.predict(X_reduced)) ** 2)
    df_reduced = X.shape[0] - (X.shape[1] - 1) - 1

    f_stat = ((ssr_reduced - ssr_full) / (df_reduced - df_full)) / (ssr_full / df_full)

    return f_stat


def exclusion_loop(dataframe):
    X = dataframe.drop(columns=[target])
    Y = dataframe[target]
    if X.columns.size < 5:
        return dataframe
    else:
        _, to_delete = min([(partial_f_test(X, Y, column), column) for column in X.columns])
        return exclusion_loop(dataframe.drop(columns=[to_delete]))


def choosing_factors(dataframe, how_many):
    X = dataframe.drop(columns=[target])
    Y = dataframe[target]
    cols = X.columns
    for to_choose in itertools.combinations(cols, how_many):
        d_frame = X[list(to_choose)]
        if check_multicollinearity(d_frame, False):
            d_frame.insert(len(d_frame.columns), target, Y)
            return d_frame
    return None


excluded = choosing_factors(df, 5)
_ = check_multicollinearity(excluded)
excluded

Наблюдаемая статистика: 2.193
Критическая статистика: 2.204
Мультиколлинеарность: отсутствует


,const,Square_Feet,Num_Bedrooms,Has_Garden,Location_Score,Price
0,1,143.635030,1,1,8.297631,602134.816747
1,1,287.678577,1,0,6.061466,591425.135386
2,1,232.998485,1,1,2.911442,464478.696880
3,1,199.664621,5,0,2.070949,583105.655996
4,1,89.004660,4,1,1.523278,619879.142523
...,...,...,...,...,...,...
495,1,138.338057,2,1,4.296086,488496.350722
496,1,195.914028,2,0,7.406261,657736.921717
497,1,69.433659,1,0,8.629724,405324.950201
498,1,293.598702,5,1,5.318891,773035.968028


In [393]:
excluded_model = sm.OLS(excluded[target], excluded.drop(columns=[target])).fit()
print(f"Средняя ошибка аппроксимации: {np.mean(np.abs(excluded_model.resid) / excluded[target].to_numpy()) * 100:.3f}%")
excluded_model.summary()

Средняя ошибка аппроксимации: 10.204%


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.679
Method:                 Least Squares   F-statistic:                     264.5
Date:                Wed, 26 Nov 2025   Prob (F-statistic):          2.11e-121
Time:                        17:09:47   Log-Likelihood:                -6280.1
No. Observations:                 500   AIC:                         1.257e+04
Df Residuals:                     495   BIC:                         1.259e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.317e+05   1.23e+04     18.788      0.000    2.07e+05    2.56e+05
Square_Feet      968.4571     41.628     23.264      0.000     886.667    1050.247
Num_Bedrooms    5.022e+04   2156.199     23.291      0.000     4.6e+04    5.45e+04
Has_Garden      2.394e+04   6217.330      3.850      0.000    1.17e+04    3.62e+04
Location_Score  3879.1553   1088.201      3.565      0.000    1741.093    6017.218
==============================================================================
Omnibus:                        8.700   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.013   Jarque-Bera (JB):                5.266
Skew:                           0.031   Prob(JB):                       0.0719
Kurtosis:                       2.501   Cond. No.                         768.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [394]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

exog = df.drop(columns=[target])
endog = df[target]

for col in exog.columns:
    exog[col] = StandardScaler().fit_transform(exog[[col]])

exog = exog.drop(columns=['const'])
component_df = pandas.DataFrame(PCA(n_components=None).fit_transform(exog))
component_df.insert(len(component_df.columns), target, endog)
component_df[target] = StandardScaler().fit_transform(component_df[[target]])
component_df

,0,1,2,3,4,5,6,7,8,9,Price
0,-1.120028,-0.602276,-1.073075,0.389201,-0.622453,1.090132,0.407359,2.155451,-0.132130,-1.462398,0.163119
1,-0.924710,1.771642,-1.004703,0.444237,-0.994907,0.469142,-0.069342,-1.161810,-0.156626,-0.707861,0.075444
2,-0.246816,-0.316409,-2.296609,-0.306740,1.257971,-1.048946,0.058903,-0.117603,-1.191538,0.386537,-0.963814
3,0.257959,0.252345,0.211960,-0.977851,0.668291,-2.019198,-0.669388,-0.645317,0.774771,1.003096,0.007335
4,0.163049,-1.372049,0.378526,-1.227521,0.703092,0.448636,1.606349,1.363090,0.227108,1.351714,0.308385
...,...,...,...,...,...,...,...,...,...,...,...
495,0.069018,-1.386598,-0.499072,-0.327594,0.880738,-0.296240,-0.381545,-0.207868,-1.261964,0.155948,-0.767192
496,-2.428152,0.379633,-0.098127,0.209144,-1.248180,-0.001977,0.501698,-0.392753,0.023481,-1.444463,0.618311
497,-0.067935,-1.821253,0.457014,-0.046231,-1.701436,0.410181,-1.630804,-0.708342,-1.540363,0.344276,-1.448081
498,2.078783,-0.055286,-0.544165,0.164328,0.791414,0.688772,0.067260,-0.059567,2.483368,0.209519,1.562216


In [395]:
component_model = sm.OLS(component_df[target], component_df.drop(columns=[target])).fit()
print(
    f"Средняя ошибка аппроксимации: {np.mean(np.abs(component_model.resid) / component_df[target].to_numpy()) * 100:.3f}%")
check_multicollinearity(component_df)
component_model.summary()

Средняя ошибка аппроксимации: 6.196%
Наблюдаемая статистика: -0.000
Критическая статистика: 33.350
Мультиколлинеарность: отсутствует


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Price   R-squared (uncentered):                   0.976
Model:                            OLS   Adj. R-squared (uncentered):              0.975
Method:                 Least Squares   F-statistic:                              1991.
Date:                Wed, 26 Nov 2025   Prob (F-statistic):                        0.00
Time:                        17:09:47   Log-Likelihood:                          222.71
No. Observations:                 500   AIC:                                     -425.4
Df Residuals:                     490   BIC:                                     -383.3
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0              0.2702      0.006     43.534      0.000       0.258       0.282
1             -0.0299      0.006     -4.669      0.000      -0.042      -0.017
2              0.0363      0.007      5.406      0.000       0.023       0.049
3             -0.3314      0.007    -47.830      0.000      -0.345      -0.318
4             -0.0538      0.007     -7.636      0.000      -0.068      -0.040
5             -0.0806      0.007    -11.262      0.000      -0.095      -0.067
6              0.5496      0.007     75.404      0.000       0.535       0.564
7             -0.1637      0.007    -22.073      0.000      -0.178      -0.149
8              0.4715      0.007     63.294      0.000       0.457       0.486
9             -0.5801      0.008    -72.860      0.000      -0.596      -0.565
==============================================================================
Omnibus:                        0.036   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.982   Jarque-Bera (JB):                0.039
Skew:                          -0.019   Prob(JB):                        0.981
Kurtosis:                       2.980   Cond. No.                         1.28
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""